< [Notebook 15](PartIV4.ipynb) | [PyFinLab Index](ALWAYS-START-HERE.ipynb) | [About this Lab Book](Welcome.ipynb) >

<a id = "ref00"></a>

<a><img src="figures/UUBS.png" width="180" height="180" border="10" /></a>

<hr>

### Notebook 16: Strategy Evaluation
In this notebook we utilise the predictive power of our multiple linear regresssion model to underpin a signal-based buy-or-sell strategy for SPY stock. We test the strategy on both training and test set data to ascertain whether or not the signal could generate profit consistently. 

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings("ignore")

### Step 0: Importing the Indexpanel data frame object

In [ ]:
# in Notebook 15 we saved the Indexpanel data frame object 
# to a CSV file for later use. Here we read it in.
Indexpanel = pd.read_csv('data/indexdata/Indexpanel.csv').set_index('Date')
Indexpanel.head()

### Step 1: Splitting the data

In [ ]:
# here we extract training and test data sets from Indexpanel
Train = Indexpanel.iloc[-2000:-1000, :]
Test = Indexpanel.iloc[-1000:, :]

### Step 2: Fitting the model to the training data

In [ ]:
# using the training data to build the model
formula = 'spy~spy_lag1+sp500+nasdaq+dji+cac40+daxi+aord+hsi+nikkei'
lm = smf.ols(formula=formula, data=Train).fit()

### Step 3: Making predictions

In [ ]:
# making predictions on both data sets
Train['PredictedY'] = lm.predict(Train)
Test['PredictedY'] = lm.predict(Test)

### Step 4: Building the strategy (and determining profit)

In [ ]:
# setting up and applying the strategy to the training data 
Train['Order'] = [1 if sig>0 else -1 for sig in Train['PredictedY']]
Train['Profit'] = Train['spy']*Train['Order']

Train['Wealth'] = Train['Profit'].cumsum()
print('Total profit made, based on applying strategy to Training data: $',
      round(Train['Profit'].sum(),2))

In [ ]:
# comparing our signal-based strategy(an active strategy) with a 
# buy-and-hold strategy(a passive strategy), for the training data
plt.figure(figsize=(10, 10))
plt.title('Performance of strategy on Train data')
plt.plot(Train['Wealth'].values, color='green', label='Signal-based strategy')
plt.plot(Train['spy'].cumsum().values, color='red', label='Buy-and-Hold strategy')
plt.legend()
plt.show()

In [ ]:
# now applying the strategy to the test data
Test['Order'] = [1 if sig>0 else -1 for sig in Test['PredictedY']]
Test['Profit'] = Test['spy']*Test['Order']

Test['Wealth'] = Test['Profit'].cumsum()
print('Total profit made, based on applying strategy to Test data: $',
      round(Test['Profit'].sum(),2))

In [ ]:
# comparing our signal-based strategy(an active strategy) with a 
# buy-and-hold strategy(a passive strategy), for the test data
plt.figure(figsize=(10, 10))
plt.title('Performance of Strategy on Test data')
plt.plot(Test['Wealth'].values, color='green', label='Signal-based strategy')
plt.plot(Test['spy'].cumsum().values, color='red', label='Buy-and-Hold strategy')
plt.legend()
plt.show()

### Step 5: Model evaluation - practical standards

We introduce two common practical standards - **Sharpe ratio**, **Maximum drawdown** to evaluate our model performance


In [ ]:
# adding in the intial share price gives us
# the value of our holding at any given time
Train['Wealth'] = Train['Wealth']+Train.loc[Train.index[0], 'Price']
Test['Wealth'] = Test['Wealth']+Test.loc[Test.index[0], 'Price']

In [ ]:
# Sharpe Ratio on Train data
Train['Return'] = np.log(Train['Wealth']) - np.log(Train['Wealth'].shift(1))
dailyr = Train['Return'].dropna()

print('Daily Sharpe Ratio is ', dailyr.mean()/dailyr.std(ddof=1))
print('Yearly Sharpe Ratio is ', (252**0.5)*dailyr.mean()/dailyr.std(ddof=1))

In [ ]:
# Sharpe Ratio in Test data
Test['Return'] = np.log(Test['Wealth']) - np.log(Test['Wealth'].shift(1))
dailyr = Test['Return'].dropna()

print('Daily Sharpe Ratio is ', dailyr.mean()/dailyr.std(ddof=1))
print('Yearly Sharpe Ratio is ', (252**0.5)*dailyr.mean()/dailyr.std(ddof=1))

In [ ]:
# maximum drawdown in Train data
Train['Peak'] = Train['Wealth'].cummax()
Train['Drawdown'] = (Train['Peak'] - Train['Wealth'])/Train['Peak']
print('Maximum Drawdown in Train is ', Train['Drawdown'].max())

In [ ]:
# maximum drawdown in Test data
Test['Peak'] = Test['Wealth'].cummax()
Test['Drawdown'] = (Test['Peak'] - Test['Wealth'])/Test['Peak']
print('Maximum Drawdown in Test is ', Test['Drawdown'].max())

< [Notebook 15](PartIV4.ipynb) | [PyFinLab Index](ALWAYS-START-HERE.ipynb) | [About this Lab Book](Welcome.ipynb) >

<div align="right"><a href="#ref00">back to top</a></div>